# Session 2: Data normalization with scVI

## Google colab setup

In [1]:
import sys
if 'google.colab' in sys.modules:
    !pip install --quiet scrnatools
    !pip install --quiet scvi-colab
    from scvi_colab import install
    install()
    from google.colab import drive
    drive.mount("/content/gdrive")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 55.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.0/103.0 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.3/330.3 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 41.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.4/647.4 kB 57.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 48.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

INFO:lightning_fabric.utilities.seed:Global seed set to 0


Mounted at /content/gdrive


In [2]:
import scrnatools as rna
import scanpy as sc
import os
import shutil
import scvi

## Data import

In [3]:
adata = sc.read("/content/gdrive/MyDrive/scrnaseq-course/data/scRNAseq_tutorial_thymus_data.h5ad")

## Setup scVI model

To train an scVI model, you first need to set up the variables you want the network to use wile learning. The main parameters to adjust are th covariates you want the model to be aware of while 'learning' a particular cell's gene expression. These include:
- Categorical covariates: these should be biologically irrelevant sample-specific nuisance variables that you want to correct for, i.e. biological replicates
- Continuous covariates: these are similar to categorical nuisance covariates, except that they take on continuous values (i.e. total counts per cell, mitochonrial read %)
- A batch key: this is the batch categorical covariate you want to use to integrate diffferent *biologically relevant* samples (i.e. WT vs KO samples)

The important distinction to make between categorical covariates and the batch key is that the batch key encodes samples you want to integrate but *expect to have meaningful biological differences*. In practce this means that by default the batch key and categorical covariates will both be accounted for/corrected in the scVI latent space and downstream dimensionality reductions (UMAPs) but only catgorical covariates' (not batch key's) effects will be removed from the normalized per-cell gene expression that is generated by scVI as well as downstream differential gene expression analysis.

In [4]:
# Setup scVI model using raw counts and batch info
scvi.model.SCVI.setup_anndata(
    adata,
    layer="raw_counts", # scVI models should always be trained on true raw counts (not log-transformed)
    categorical_covariate_keys=None,
    continuous_covariate_keys=None,
    batch_key="genotype", # The batch
)
model = scvi.model.SCVI(adata)

## Train scVI model

Training the scVI model will take a very long time unless it is run on a machine with a discrete GPU (i.e. not most laptops). It will also take longer the more genes and cells are in the dataset

In [5]:
# Train scVI model - run on google colab with a GPU session or a personal computer/server with an Nvidia GPU
model.train()

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 400/400: 100%|██████████| 400/400 [02:06<00:00,  3.99it/s, loss=1.77e+03, v_num=1]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=400` reached.


Epoch 400/400: 100%|██████████| 400/400 [02:06<00:00,  3.15it/s, loss=1.77e+03, v_num=1]


## Get scVI normalized expression/latent representation

The scVI model learns a 15-dimension latent representation of the expression data that is then used for downstream clustering and further dimensionality reduction (i.e. UMAP/TSNE). This latent representation is learned to correct all batch effects encoded by the batch_key, continuous_covariates, and categorical_covariates supplied during model setup.

In [6]:
adata.obsm["X_scVI"] = model.get_latent_representation()

scVI normalized gene expression learned during model training is encoded as a non-zero probability that corresponds to the denoised expression frequency in a given cell. This is scaled by the provided library size (usually 10,000, but this is arbitrary) to give an estimate on the number of reads of a particular gene per 10,000 total reads expected to be observed in the given cell. This normalized expression is generated based on a cell's original 'batch_key' by default, which is to say that any biological differences in gene expression across different 'batch_keys' is preserved, while differences in gene expression across different 'categorical_covariates' are lost. This means that a gene that has variability in expression acrosss WT and KO samples maintains that variability, while expression that varies across biological replicates is removed (as it iss likely due to technical noise).

In [7]:
adata.layers["scVI_normalized"] = model.get_normalized_expression(library_size=1e4)

## Saving/Loading scVI models

scVI model training is non-deterministic, so it is important to save your model immediately to allow downstream analyses to be repeatable.

In [11]:
# If a model folder with the specified path/name already exists you have to delete it before resaving the scVI model
if os.path.isdir("/content/gdrive/MyDrive/scrnaseq-course/data/scRNAseq_tutorial_thymus_data_scVI_model"):
    shutil.rmtree("/content/gdrive/MyDrive/scrnaseq-course/data/scRNAseq_tutorial_thymus_data_scVI_model")
    
model.save("/content/gdrive/MyDrive/scrnaseq-course/data/scRNAseq_tutorial_thymus_data_scVI_model")

In [12]:
model = scvi.model.SCVI.load("/content/gdrive/MyDrive/scrnaseq-course/data/scRNAseq_tutorial_thymus_data_scVI_model", adata=adata)

INFO     File /content/gdrive/MyDrive/scrnaseq-course/data/scRNAseq_tutorial_thymus_data_scVI_model/model.pt       
         already downloaded                                                                                        


In [13]:
adata.write("/content/gdrive/MyDrive/scrnaseq-course/data/scRNAseq_tutorial_thymus_data.h5ad")